# Foreground Database Import

This notebook contains the required steps to import the foreground datatabase. The database comes originally as an [excel file](Foreground.xlsx).

In [1]:
from pathlib import Path
from pprint import pprint

In [2]:
import bw2data as bd
import bw2io as bi

In [3]:
#Importing the variables with the project name and background db
from project_details import EI_DB_NAME, PROJECT_NAME

In [62]:
bd.projects.set_current(PROJECT_NAME)
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-391-cutoff

In [61]:
#to erase databases
#asphalt = bd.Database("asphalt")
#del bd.databases[asphalt.name]

In [12]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert EI_DB_NAME in bd.databases

## Extract foreground inventory data

In [64]:
# Include the path to the foreground database
FG_DB = Path("./Foreground_unique.xlsx")

# Import your LCI
lci = bi.ExcelImporter(FG_DB)

Extracted 1 worksheets in 0.04 seconds


## Transform the data from xlsx format to Brightway format

### Match the foreground database to itself

In [65]:
# Need to match FG_DB to itself
lci.match_database(fields=["name", "unit", "location"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the background db

In [66]:
# Need to match FG_DB to the biosphere
lci.match_database(EI_DB_NAME, fields=["name", "reference product", "location", "unit"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the biosphere3 database

In [67]:
# Need to match FG_DB to the biosphere
lci.match_database("biosphere3", fields=["name", "categories"])

Applying strategy: link_iterable_by_fields


In [68]:
bi.create_core_migrations()
# Once your package is imported we need to apply strategies
lci.apply_strategies()

# We need to match databases - name and categories but ATTENTION! the categories in
# the excel file is "None" so we willlci.write_excel() also need to match against unit.

lci.statistics()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 4.71 seconds
36 datasets
185 exchanges
0 unlinked exchanges
  


(36, 185, 0)

In [69]:
import_statistics = lci.statistics(print_stats=False)

# If all exchanges have been linked, proceed to write the database
if import_statistics[2] == 0:
    lci.write_database()
else:
    print(f"There are unlinked exchanges {import_statistics}")
    lci.write_excel()
    for unlinked_exchange in lci.unlinked:
        pprint(unlinked_exchange)

bd.databases

100%|███████████████████████████████████████████████████| 36/36 [00:00<00:00, 6054.57it/s]


Vacuuming database 
Created database: asphalt


Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

## DB structure checks

In [70]:
import bw2analyzer as bwa

In [71]:
FG_DB_NAME = "asphalt"

In [72]:
pavement_complete_a = bd.Database(FG_DB_NAME).get("DZOAB, A")

In [73]:
bwa.print_recursive_supply_chain(pavement_complete_a, max_level=1)

1: 'DZOAB, A' (kilogram, NL, None)
  1: 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)
  1: 'A1, pavement, materials, A (own material)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (bitumen)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (crushed stone)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (own material)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (crushed sand)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (drip resistant material)' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (filler)' (kilogram, NL, None)
  1: 'A3, pavement, production, A' (kilogram, NL, None)
  1: 'A4, pavement, transport to site' (kilogram, NL, None)
  1: 'A5, pavement, construction' (kilogram, NL, None)
  1: 'C1, pavement, demolition' (kilogram, NL, None)
  1: 'C2, pavement, transport to processing' (kilogram, NL, None)
  1: 'C3, pavement, processing (crane and shovel)' (ki

In [74]:
bwa.print_recursive_supply_chain(pavement_complete_a, max_level=2)

1: 'DZOAB, A' (kilogram, NL, None)
  1: 'A1, pavement, materials, A (excl. own material)' (kilogram, NL, None)
    0.3: 'asphalt granulate, free of burden' (kilogram, NL, None)
    0.0412: 'bitumen adhesive compound production, hot' (kilogram, RER, None)
    0.586: 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None)
    0.0342: 'gravel production, crushed' (kilogram, RoW, None)
    0.0021: 'cellulose fibre production (without borax and boric acid)' (kilogram, RoW, None)
    0.027: 'medium filler' (kilogram, NL, None)
  1: 'A1, pavement, materials, A (own material)' (kilogram, NL, None)
    0.0094: 'crushed stone, from quarry in Europe, excluding transport to the Netherlands' (kilogram, NL, None)
  1: 'A2, pavement, transport to plant, A (bitumen)' (kilogram, NL, None)
    0.0103: 'market for transport, freight, lorry, unspecified' (ton kilometer, RER, None)
  1: 'A2, pavement, transport to plant, A (crushed stone)' (kilogram, NL, None)
  